In [1]:
import pandas as pd
import glob
import numpy as np
import os

In [2]:
defile_list_test=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/*.csv")
defile_list_test=[x for x in defile_list_test if "/T_Quadrant" in x]
all_test=pd.DataFrame()
for file_path in defile_list_test:
    df=pd.read_csv(file_path,dtype=str)
    all_test=all_test.append(df)

In [3]:
all_test.shape

(18154859, 3)

In [4]:
df_decile_test=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/all_test_no_others2019-01-18_21:50:51.386367.csv",usecols=['frmindex','customer_id_hashed'],dtype=str)

df_decile_test.shape

(18154859, 2)

In [5]:
all_test=pd.merge(all_test,df_decile_test,on='customer_id_hashed')
all_test.shape


(18154859, 4)

In [6]:
import random
random.seed(1)
total_rows=len(all_test)
df_control_from_previous_test=pd.DataFrame()

for ind,group in all_test.groupby(['segment','frmindex']):
    random_list=random.sample(range(len(group)), int(np.round(len(group)/total_rows*400000)))

    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    df_control=group[group['index'].isin(random_list)]
    df_control_from_previous_test=df_control_from_previous_test.append(df_control)
df_control_from_previous_test.shape    

(399998, 5)

In [7]:
output_df_control_from_previous_test=df_control_from_previous_test[['customer_id_hashed','email_address_hash','segment']]
output_df_control_from_previous_test['segment']=output_df_control_from_previous_test['segment']+"_ControlInT"

os.getcwd()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/Pick_Control_within_Test'

In [8]:
output_df_control_from_previous_test.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/Pick_Control_within_Test/output_control_with_in_Q1_test.csv",index=False)
df_control_from_previous_test.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/Pick_Control_within_Test/Control_with_in_Q1_test_with_decile.csv",index=False)

In [9]:
output_df_control_from_previous_test.head(2)

,customer_id_hashed,email_address_hash,segment
116,12856583db4dbb005e5de4b6991934872a6a7fcb3467b0...,02ef1144e18c2dffa9231f010867716b5a9755a3b2d815...,T_Quadrant III_P_H_2019Q1_ControlInT
176,5992b52cac7716d755901d7c9b21782df61634b4da51dd...,c6a42afe8a2ba122688724f95c25cb1dfb76133542c5f1...,T_Quadrant III_P_H_2019Q1_ControlInT


In [10]:
len(output_df_control_from_previous_test['segment'].unique())

36

In [11]:
summary_control_in_test=output_df_control_from_previous_test.groupby(['segment'])['customer_id_hashed'].count().to_frame().reset_index()
summary_control_in_test.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/Pick_Control_within_Test/summary_output_control_with_in_Q1_test.csv",index=False)
